# 108-2 Data Mining
## Homework #3 : Association Rule
## Instructor: Hsueh-Chan Lu
## Submitted by: Carlo D. Pastoral
## Student ID: N26087048

- In this homework, the task is to pick a multi-value attribute from my data and analyze the association relationships between those values.

- To do the task, I am going to use python as my programming language.
- To strat with, I am going to import the essential packages for exploring my datatset.

In [61]:
import pandas as pd 
import numpy as np
import json
import pprint
from scipy import stats
from collections import  Counter
from scipy.stats import entropy
pd.set_option('display.max_rows', None)


- Now lets print a 10 random sample from the daset and pick an attribute with multi-value.

In [62]:
data = pd.read_csv('Data/USvideos.csv')
sample = data.sample(n=50)
sample


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
40390,T_o9bDu_srU,18.12.06,Marshmello - TELL ME,Marshmello,10,2018-06-08T04:00:03.000Z,"marshmello|""Tell Me""|""marshmello tell me""|""marshmallow""|""you and me""|""keep it mello""|""dance""|""tomorrowland""|""Marshmello friends""|""tell me marshmello""|""new marshmello""|""marshmello alone""|""alone marshmello""|""marshmello summer""|""EDC""|""EDM""|""MArshmello EDC""|""alan walker""|""NCS""|""Monstercat""|""dame tu cosita""|""cooking with Marshmello""|""Skrillex""|""Diplo""|""Calvin harris""|""alone""|""маршмеллоу""|""ninja""|""machimelo""|""friends""|""trap music""|""fly""|""joytime""|""mello""|""fortnite""|""gaming music""|""e3""|""jauz""|""slushii""|""ookay""|""ghastly""|""drake""|""kanye""|""pop""|""bass""",1584165,106988,2577,14289,https://i.ytimg.com/vi/T_o9bDu_srU/default.jpg,False,False,False,Marshmello - TELL MEStream 'TELL ME' ▶ http://smarturl.it/MarshmelloTellMeGet your NEW Mello™️ by Marshmello gear HERE ▶ http://shop.marshmellomusic.comWatch Cooking with Marshmello HERE ▶ http://youtube.com/playlist?list=PLcYK4PlHbZQtXROf5fnrr4dO4ruWiv7tsSUBSCRIBE HERE ▶ http://youtube.com/marshmellomusic?sub_confirmation=1WATCH YOU CAN CRY MUSIC VIDEO ▶ https://youtu.be/2SJ0hgdciNEWATCH EVERYDAY MUSIC VIDEO ▶ https://youtu.be/bEdcJY8Emm8WATCH FLY MUSIC VIDEO ▶ https://youtu.be/oRArmtMA9AIWATCH FRIENDS MUSIC VIDEO ▶ https://youtu.be/jzD_yyEcp0MWATCH SPOTLIGHT MUSIC VIDEO ▶ https://youtu.be/7R1N-8SoqcMWATCH LOVE U MUSIC VIDEO ▶ https://youtu.be/D-pKeb6Wf4UWATCH TAKE IT BACK MUSIC VIDEO ▶ https://youtu.be/P9Ijqa_2eu0WATCH SILENCE MUSIC VIDEO ▶ https://youtu.be/Tx1sqYc3qasWATCH BLOCKS MUSIC VIDEO ▶ https://youtu.be/5E4ZBSInqUUWATCH YOU & ME MUSIC VIDEO ▶ https://youtu.be/fiusxyygqGkWATCH FIND ME MUSIC VIDEO ▶ https://youtu.be/ymq1WdGUcw8WATCH MOVING ON MUSIC VIDEO ▶ https://youtu.be/yU0tnrEk8H4WATCH SUMMER MUSIC VIDEO ▶ https://youtu.be/2vMH8lITTCEWATCH ALONE MUSIC VIDEO ▶ https://youtu.be/ALZHF5UqnU4WATCH KEEP IT MELLO MUSIC VIDEO ▶ https://youtu.be/_J_VpmXAzqgMARSHMELLO: Spotify | http://spoti.fi/MarshmelloSoundCloud | http://soundcloud.com/marshmellomusicInstagram | http://instagram.com/marshmellomusicTwitter | http://twitter.com/marshmellomusicFacebook | http://facebook.com/marshmellomusicMerch | http://shop.marshmellomusic.com/Produced by Toon53 Productions | https://www.toon53productions.com
31055,1FJD7jZqZEk,18.26.04,Jurassic World: Fallen Kingdom - Final Trailer [HD],Universal Pictures,24,2018-04-18T15:00:10.000Z,[none],12916876,177614,7415,17481,https://i.ytimg.com/vi/1FJD7jZqZEk/default.jpg,False,False,False,"Jurassic World: Fallen Kingdom \nIn Theaters June 22, 2018 \nhttps://www.jurassicworld.com \n\nIt’s been four years since theme park and luxury resort Jurassic World was destroyed by dinosaurs out of containment. Isla Nublar now sits abandoned by humans while the surviving dinosaurs fend for themselves in the jungles. \n\nWhen the island’s dormant volcano begins roaring to life, Owen (Chris Pratt) and Claire (Bryce Dallas Howard) mount a campaign to rescue the remaining dinosaurs from this extinction-level event. Owen is driven to find Blue, his lead raptor who’s still missing in the wild, and Claire has grown a respect for these creatures she now makes her mission. Arriving on the unstable island as lava begins raining down, their expedition uncovers a conspiracy that could return our entire planet to a perilous order not seen since prehistoric times.\n\nWith all of the wonder, adventure and thrills synonymous with one of the most popular and successful series in cinema history, this all-new motion-picture event sees the return of favorite characters and dinosaurs—along with new breeds more awe-inspiring and terrifying than ever before. Welcome to Jurassic World: Fallen Kingdom. \n\nStars Pratt and Howard return alongside 

- From the 10 random samples, I saw that the "Tag" column is a multi-value attribute and it will be a good candidate for analyzing the association relationships between the tags. 
- However there is just one problem, **It is dirty.** 
- So first we need to clean this multi-value attribute first..

- To do this, first let us create a new dataframe container called **new_df** and put the 'video_id', 'tags', and 'views' cloumns here. 

In [63]:
new_df = pd.DataFrame(sample,columns = ['video_id','tags','views'])

 - Here I defined a funtion for cleaning every entry in **tags** column and converting each as list of tags while storing all in another list so I have a list of lists. Later on I will use this for applying Apriori.

 - I also changed the content of the **tags** column in the **new_df** dataframe with the cleaned one.

In [64]:
def cleanString(incomingString):
    newstring = incomingString
    newstring = newstring.replace("\"","")
    newstring = list(filter(None,newstring.split('|')))
    return newstring

tagsList = []
tagsList = [ cleanString(col['tags']) for idx,col in new_df.iterrows()] 
new_df['tags'] = tagsList
new_df


,video_id,tags,views
40390,T_o9bDu_srU,"[marshmello, Tell Me, marshmello tell me, marshmallow, you and me, keep it mello, dance, tomorrowland, Marshmello friends, tell me marshmello, new marshmello, marshmello alone, alone marshmello, marshmello summer, EDC, EDM, MArshmello EDC, alan walker, NCS, Monstercat, dame tu cosita, cooking with Marshmello, Skrillex, Diplo, Calvin harris, alone, маршмеллоу, ninja, machimelo, friends, trap music, fly, joytime, mello, fortnite, gaming music, e3, jauz, slushii, ookay, ghastly, drake, kanye, pop, bass]",1584165
31055,1FJD7jZqZEk,[[none]],12916876
9605,MMu-_KRZhwM,"[Collegehumor, CH originals, comedy, sketch comedy, internet, humor, funny, sketch, holidays, new year’s eve, advice, lists, partying, awkward, being single, FML, cringeworthy, desperate, fails, well that’s disappointing, fomo, thirsty, siobhan thompson, CH Shorts, latest]",555596
14272,gTRRvvE3HiU,"[madelaine, madelainepetsch, madelaine petsch, petsch, cheryl, blossom, cherylblossom, cheryl blossom, riverdale, shane dawson, parody, riverdale parody]",311179
22979,JKP-rOESfMs,"[Super Smash Bros, Nintendo, Switch, Wii U, 3DS, Link, Breath of the Wild, Mario, Super Mario Odyssey, Donkey Kong, Pit, Kid Icarus, Captain Falcon, Metroid, Samus, Pokemon, Pokemon Trainer, Red, Pikachu, Jiggilypuff, Ness, Discussion, Characters, Update, News, Preview, Flames, Logo, Yoshi, Peach, Bowser, Kirby, Star Allies, Dedede, Meta Knight]",768838
37460,D6Oy4LfoqsU,"[I gave safiya nygaard a perfect hair makeover based on her features: bts, brad mondo, safiya and tyler, safiya nygaard, hair transformation, makeover, I got a perfect makeover based on my features, bts, hairdresser reacts, before and after, hair, makeup, transformation, ANTM, what not to wear, the ideal haircut and color for your face, safiya buzzfeed, color for your skin tone, haircut for your face shape, tutorial, balayage, hair stylist, hair color, hair tutorial]",966340
19747,J6Tc4iyefe0,"[nbc news, breaking news, us news, world news, politics, nightly news, current events, top stories, school shooting, florida school shooting, parkland, florida, high school shooting, shooting, marjory stoneman douglas high school, high school, school, broward county, victims, nikolas, mass shooting, florida shooting, parkland florida, nikolaus, nicholas cruz, broward, first responder, gunshot, mass murder, school shooting in florida, coral springs police, broward country police]",63358
35455,TuPT8ZF0Vl4,"[fox, fox sports, fs1, fox sports 1, undisputed, skip bayless, shannon sharpe, skip and shannon, fs1 undisputed, skip, bayless, shannon, sharpe, joy taylor, sports, news, nba, basketball, 2018 nba playoffs, nba playoffs, nba playoffs 2018, golden state, warriors, golden state warriors, houston, rockets, houston rockets, 2018 Western Conference, Warriors in the West, 2018 NBA, 2018 Western Conference Finals]",1018499
37779,JyG5hdbQpDM,"[kim kardashian, kkw beauty, everyday makeup, amber scholl, tutorial, how to, highlight, natural, easy, first impressions, review]",841502
27425,S9KxqRUcnCU,"[dude perfect, dude perfect stereotypes, dude perfect water bottle flip, bottle flip, water bottle flip, dude perfect bottle flip, dude perfect basketball, dp, dude perfect world record, edition, nerf, trick shots, trick shot, family, ping pong, bowling, clean, family friendly, bubble wrap, soccer, football, spinner, spinners, fidget spinners, dude, boomerang, throw, thrower, logan, broadbent]",20462695


- Now that I have already prepared the data, I can now preceed in finding the association rules.
- To do that I will be using a python package called **apyori** to use its apriori function.
- Its **apriori** function only requires minimum values for **support**, **confidence**, and **lift**.
- It also allows the user to define a minimum number of words that will be scanned for antecedent and consequent.

In [65]:
from apyori import apriori

\begin{equation*}
Support(\{{{X}\}}\rightarrow\{{{Y}\}}) = \frac{Number\;of\;transactions\;containing\;both\;X\;and\;Y}{Total\;number\;of\;transactions}
\end{equation*}

\begin{equation*}
Confidence(\{{{X}\}}\rightarrow\{{{Y}\}}) = \frac{Number\;of\;transactions\;containing\;both\;X\;and\;Y}{Number\;of\;transactions\;containting X}
\end{equation*}

\begin{equation*}
Lift(\{{{X}\}}\rightarrow\{{{Y}\}}) = \frac{\frac{Number\;of\;transactions\;containing\;both\;X\;and\;Y}{Number\;of\;transactions\;containting\;X}}{Fraction\;of\;transactions\;containing\;Y}
\end{equation*}

- Now, to define a minimum values for **support**, **confidence**, and **lift** I will follow the formulas above. 
- I would like to set the minimum support to 2/50 so that I will get those Xs and Ys that occured twice over the fifty samples.
- For the confidence I will set it to ratio of 1/4. 
- For the minimum lift, I set it to 10 to limit the popularity of Ys  

In [71]:
association_rules = list(apriori(tagsList,min_support = 3/50, min_confidence = .25,min_lift = 10, min_length = 3))
len(association_rules)

16

- The function the I have used above returns a list of association rules. 
- And by getting the length of this list, I see that I've got a list of 16 association rules.
- To print the results I need to iterate through this list.

In [67]:

for item in association_rules:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")


Rule: family friendly -> bottle flip
Support: 0.06
Confidence: 1.0
Lift: 12.5
Rule: trick shots -> bottle flip
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: funny video -> celebrities
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: humor -> celebrities
Support: 0.06
Confidence: 1.0
Lift: 10.0
Rule: jokes -> celebrities
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: family friendly -> trick shots
Support: 0.06
Confidence: 0.75
Lift: 12.5
Rule: football -> soccer
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: humor -> funny video
Support: 0.06
Confidence: 1.0
Lift: 10.0
Rule: funny video -> jokes
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: humor -> jokes
Support: 0.06
Confidence: 0.6
Lift: 10.0
Rule: family friendly -> bottle flip
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: humor -> funny video
Support: 0.06
Confidence: 1.0
Lift: 16.666666666666668
Rule: funny video -> jokes
Support: 0.06
Confidence

- We can see that from these associatin rules, most are pretty obvious such **football -> soccer**. 
- We can also notice that most of the confidence value are equal to one but we can also notice that those association rule that has confidence value of less than 1,   the relativeness of antecedent and consequent is questionable. 
- Let us see for example the **Rule: family friendly -> **trick shots**, it has a support value of 0.06 which means that it **family friendly** appeared three times as with the **trick shots**  in the data. 
- And, it has a confidence value of 0.75 which means that it **trick shots** tag is more likely to use together with **family friendly** but, we intuitively know that these two tags have a weak association and there might be misleading about its confidence value. 
- So, to verify the association between these two tags we are going to look at the **lift** value. 
- We can see that the lift value for this association rule is **12.5**. Since the value is greatly more than 1, now this verifies that having a **family friendly** tag has a high chance of also having the **trick shots** tag in the video post.  